In [1]:
import subprocess
from urllib.parse import urljoin
from io import StringIO

from dateutil.parser import parse as parse_datetime
import pandas as pd
import requests
from lxml import html
import js2xml

from helpers import derive_ein_from_filename

In [2]:
TARGET_COLUMNS = [
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'alt_hcpcs_cpt',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [3]:
transparency_page = "https://www.adventisthealth.org/patient-resources/financial-services/healthcare-costs-and-charges/cost-estimator-tool/"
ccn = "050455"
app_url = "https://apps.para-hcfs.com/PTT/FinalLinks/Adventist_Bakersfield_V3.aspx"
#ccn = "051317"
#app_url = "https://apps.para-hcfs.com/PTT/FinalLinks/Adventist_Clear_Lake_V3.aspx"

In [4]:
resp = requests.get(app_url)
resp

<Response [200]>

In [5]:
tree = html.fromstring(resp.text)
js_link = tree.xpath('//script[contains(@src, "/PTT/extnet/extnet-init-js")]/@src')[0]
js_url = urljoin(resp.url, js_link)
resp1 = requests.get(js_url)
parsed = js2xml.parse(resp1.text)
db_name = parsed.xpath('//object[./property/string[text()="App.hdnDB_Container"]]/property[@name="value"]/string/text()')[0]
db_name

'dbAHBBAKERSFIELDCA'

In [6]:
params = {
    'dbName': db_name,
    'type': 'CDMWithoutLabel'
}

data = '------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__EVENTTARGET"\r\n\r\nResourceManager\r\n------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__EVENTARGUMENT"\r\n\r\n-|public|DownloadReport\r\n------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__ExtNetDirectEventMarker"\r\n\r\ndelta=true\r\n------WebKitFormBoundarysVqstz3xq11k5yBT--\r\n'

resp2 = requests.post('https://apps.para-hcfs.com/PTT/FinalLinks/Reports.aspx', params=params, data=data)
print(resp2)

<Response [200]>


In [7]:
filename = resp2.headers['content-disposition'].split('"')[1]
filename

'952294234_adventist-health---bakersfield_standardcharges.csv'

In [8]:
chunks = resp2.text.split("\r\n\r\n")
len(chunks)

4

In [9]:
chunks[0]

'Run Date: 5/30/2023 8:19:19 PM\r\nCDM Date Stamp: 4/20/2023'

In [10]:
date_str = chunks[0].split(" ")[-1]
date_str

last_updated = parse_datetime(date_str).isoformat().split("T")[0]
last_updated

'2023-04-20'

In [11]:
chunks[1][:100]

'Procedure Code,Procedure Description,Price Tier,Revenue Code,CPT HCPCS Code,Modifier1,Modifier2,NDC '

In [12]:
chunks[2][:100]

'Diagnosis Related Group Code,Diagnosis Related Group Description,Price Tier,Revenue Code,CPT HCPCS D'

In [13]:
csv_f = open(filename, "w")
csv_f.write(resp2.text)
csv_f.close()

In [14]:
csv_buf = StringIO(chunks[2])

df_in = pd.read_csv(csv_buf, dtype=str)
df_in

,Diagnosis Related Group Code,Diagnosis Related Group Description,Price Tier,Revenue Code,CPT HCPCS DRG Code,Modifier1,Modifier2,NDC Code,Rx Unit Multiplier,Gross Charge,...,KAISER_PPO_POS_-_ALL_OTHER_PLANS,KERN_LEGACY_HP_EPO_-_ALL_OTHER_PLANS,KERN_LEGACY_SHARE_SELECT,OSCAR_-_ALL_PLANS,PHCS-_ALL_PLANS,THREE_RIVERS_PROVIDER_NETWORK-_ALL_PLANS,UHC_ALL_PAYER_-_ALL_OTHER_PLANS,UHC_HMO,UHC_JLL_CSP,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS
0,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,Inpatient,NaN,003,NaN,NaN,NaN,NaN,1399304.2366,...,853575.58,248564.97,229744.97,274067.61,208400.00,1119443.39,413338.13,313857.60,392656.39,481583.25
1,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,Inpatient,NaN,004,NaN,NaN,NaN,NaN,908244.2900,...,554029.02,99750.00,89970.00,185966.08,105000.00,726595.43,244932.33,182659.07,232683.66,230870.71
2,020,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,Inpatient,NaN,020,NaN,NaN,NaN,NaN,582781.5800,...,355496.76,25057.76,23044.76,125993.01,69300.00,466225.26,165942.96,123752.50,157644.42,160609.09
3,023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,Inpatient,NaN,023,NaN,NaN,NaN,NaN,294478.6000,...,179631.94,36708.96,34695.96,77619.38,30900.00,235582.88,111929.49,84510.61,106329.97,76049.66
4,024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,Inpatient,NaN,024,NaN,NaN,NaN,NaN,151325.3124,...,92308.44,34433.50,32420.50,53477.93,16250.00,121060.25,80220.20,60872.61,76206.39,44663.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,981,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,981,NaN,NaN,NaN,NaN,214753.0230,...,130999.34,38941.50,36461.50,62031.58,29400.00,171802.42,87487.46,65863.90,83111.10,55056.51
465,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,982,NaN,NaN,NaN,NaN,193489.6350,...,118028.68,33467.75,30667.75,33968.13,31750.00,154791.71,44738.76,33364.08,42501.45,58884.45
466,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,983,NaN,NaN,NaN,NaN,112140.1000,...,68405.46,2500.00,2250.00,22619.23,3100.00,89712.08,29791.36,22217.00,28301.54,4648.00
467,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Inpatient,NaN,987,NaN,NaN,NaN,NaN,166288.4179,...,101435.93,31834.01,29604.01,44940.53,22450.00,133030.73,59190.30,44141.36,56230.29,47451.02


In [15]:
df_mid = pd.DataFrame(df_in)
df_mid = df_mid.rename(columns={
    'Procedure Code': 'local_code',
    'Procedure Description': 'description',
    'Price Tier': 'setting',
    'Revenue Code': 'rev_code',
    'CPT HCPCS Code': 'hcpcs_cpt',
    'NDC Code': 'ndc',
    'Rx Unit Multiplier': 'drug_hcpcs_multiplier',
    'Modifier1': 'modifiers',
    'Diagnosis Related Group Code': 'ms_drg',
    'Diagnosis Related Group Description': 'description',
    'CPT HCPCS DRG Code': 'code',
    'Shoppable Services Code': 'local_code',
    'Shoppable Services Description': 'description'
})

df_mid

,ms_drg,description,setting,rev_code,code,modifiers,Modifier2,ndc,drug_hcpcs_multiplier,Gross Charge,...,KAISER_PPO_POS_-_ALL_OTHER_PLANS,KERN_LEGACY_HP_EPO_-_ALL_OTHER_PLANS,KERN_LEGACY_SHARE_SELECT,OSCAR_-_ALL_PLANS,PHCS-_ALL_PLANS,THREE_RIVERS_PROVIDER_NETWORK-_ALL_PLANS,UHC_ALL_PAYER_-_ALL_OTHER_PLANS,UHC_HMO,UHC_JLL_CSP,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS
0,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,Inpatient,NaN,003,NaN,NaN,NaN,NaN,1399304.2366,...,853575.58,248564.97,229744.97,274067.61,208400.00,1119443.39,413338.13,313857.60,392656.39,481583.25
1,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,Inpatient,NaN,004,NaN,NaN,NaN,NaN,908244.2900,...,554029.02,99750.00,89970.00,185966.08,105000.00,726595.43,244932.33,182659.07,232683.66,230870.71
2,020,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,Inpatient,NaN,020,NaN,NaN,NaN,NaN,582781.5800,...,355496.76,25057.76,23044.76,125993.01,69300.00,466225.26,165942.96,123752.50,157644.42,160609.09
3,023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,Inpatient,NaN,023,NaN,NaN,NaN,NaN,294478.6000,...,179631.94,36708.96,34695.96,77619.38,30900.00,235582.88,111929.49,84510.61,106329.97,76049.66
4,024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,Inpatient,NaN,024,NaN,NaN,NaN,NaN,151325.3124,...,92308.44,34433.50,32420.50,53477.93,16250.00,121060.25,80220.20,60872.61,76206.39,44663.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,981,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,981,NaN,NaN,NaN,NaN,214753.0230,...,130999.34,38941.50,36461.50,62031.58,29400.00,171802.42,87487.46,65863.90,83111.10,55056.51
465,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,982,NaN,NaN,NaN,NaN,193489.6350,...,118028.68,33467.75,30667.75,33968.13,31750.00,154791.71,44738.76,33364.08,42501.45,58884.45
466,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,983,NaN,NaN,NaN,NaN,112140.1000,...,68405.46,2500.00,2250.00,22619.23,3100.00,89712.08,29791.36,22217.00,28301.54,4648.00
467,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Inpatient,NaN,987,NaN,NaN,NaN,NaN,166288.4179,...,101435.93,31834.01,29604.01,44940.53,22450.00,133030.73,59190.30,44141.36,56230.29,47451.02


In [16]:
def unify_modifiers(m1, m2):
    if m1 is None:
        return None

    if m1 is not None:
        if m2 is not None:
            return m1 + "|" + m2

        return m1

assert unify_modifiers(None, None) is None
assert unify_modifiers("TC", None) == "TC"
assert unify_modifiers("TC", "TC") == "TC|TC"

In [17]:
df_mid.loc[df_mid['modifiers'].isnull(), 'modifiers'] = None
df_mid.loc[df_mid['Modifier2'].isnull(), 'Modifier2'] = None

df_mid['modifiers'] = df_mid[['modifiers', 'Modifier2']].apply(lambda row: unify_modifiers(row['modifiers'], row['Modifier2']), axis=1)
del df_mid['Modifier2']
df_mid

,ms_drg,description,setting,rev_code,code,modifiers,ndc,drug_hcpcs_multiplier,Gross Charge,Discounted Cash Price,...,KAISER_PPO_POS_-_ALL_OTHER_PLANS,KERN_LEGACY_HP_EPO_-_ALL_OTHER_PLANS,KERN_LEGACY_SHARE_SELECT,OSCAR_-_ALL_PLANS,PHCS-_ALL_PLANS,THREE_RIVERS_PROVIDER_NETWORK-_ALL_PLANS,UHC_ALL_PAYER_-_ALL_OTHER_PLANS,UHC_HMO,UHC_JLL_CSP,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS
0,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,Inpatient,NaN,003,None,NaN,NaN,1399304.2366,265867.81,...,853575.58,248564.97,229744.97,274067.61,208400.00,1119443.39,413338.13,313857.60,392656.39,481583.25
1,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,Inpatient,NaN,004,None,NaN,NaN,908244.2900,172566.42,...,554029.02,99750.00,89970.00,185966.08,105000.00,726595.43,244932.33,182659.07,232683.66,230870.71
2,020,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,Inpatient,NaN,020,None,NaN,NaN,582781.5800,110728.50,...,355496.76,25057.76,23044.76,125993.01,69300.00,466225.26,165942.96,123752.50,157644.42,160609.09
3,023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,Inpatient,NaN,023,None,NaN,NaN,294478.6000,55950.93,...,179631.94,36708.96,34695.96,77619.38,30900.00,235582.88,111929.49,84510.61,106329.97,76049.66
4,024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,Inpatient,NaN,024,None,NaN,NaN,151325.3124,28751.81,...,92308.44,34433.50,32420.50,53477.93,16250.00,121060.25,80220.20,60872.61,76206.39,44663.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,981,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,981,None,NaN,NaN,214753.0230,40803.07,...,130999.34,38941.50,36461.50,62031.58,29400.00,171802.42,87487.46,65863.90,83111.10,55056.51
465,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,982,None,NaN,NaN,193489.6350,36763.03,...,118028.68,33467.75,30667.75,33968.13,31750.00,154791.71,44738.76,33364.08,42501.45,58884.45
466,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,983,None,NaN,NaN,112140.1000,21306.62,...,68405.46,2500.00,2250.00,22619.23,3100.00,89712.08,29791.36,22217.00,28301.54,4648.00
467,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Inpatient,NaN,987,None,NaN,NaN,166288.4179,31594.80,...,101435.93,31834.01,29604.01,44940.53,22450.00,133030.73,59190.30,44141.36,56230.29,47451.02


In [18]:
money_columns = df_mid.columns.to_list()[8:]
money_columns

['Gross Charge',
 'Discounted Cash Price',
 'De-identified minimum negotiated charge',
 'De-identified maximum negotiated charge',
 'AETNA-_ALL_PLANS',
 'AFFILIATED_HEALTH_FUNDS-ALL_PLANS',
 'BEECH_STREET-_ALL_PLANS',
 'BLUE_CROSS_MCS_-_ALL_OTHER_PLANS',
 'BLUE_CROSS_NON-MCS',
 'BLUE_SHIELD_EPN',
 'BLUE_SHIELD_EPO_PPO',
 'BLUE_SHIELD_HMO_POS_-_ALL_OTHER_PLANS',
 'CIGNA-_ALL_OTHER_PLANS',
 'CIGNA_HMO_OPEN_ACCESS',
 'FIRST_HEALTH_COVENTRY-_ALL_PLANS',
 'GALAXY_HEALTH_NETWORK-_ALL_PLANS',
 'GEM_CARE-_ALL_PLANS',
 'HEALTH_NET_HMO_PPO_-_ALL_PLANS',
 'INTEGRATED_HEALTH_PLAN-_ALL_PLANS',
 'INTERPLAN-_ALL_PLANS',
 'KAISER_COMM_HMO_',
 'KAISER_NON-KP',
 'KAISER_PPO_POS_-_ALL_OTHER_PLANS',
 'KERN_LEGACY_HP_EPO_-_ALL_OTHER_PLANS',
 'KERN_LEGACY_SHARE_SELECT',
 'OSCAR_-_ALL_PLANS',
 'PHCS-_ALL_PLANS',
 'THREE_RIVERS_PROVIDER_NETWORK-_ALL_PLANS',
 'UHC_ALL_PAYER_-_ALL_OTHER_PLANS',
 'UHC_HMO',
 'UHC_JLL_CSP',
 'WESTERN_GROWERS_ASSURANCE-_ALL_PLANS']

In [19]:
remaining_columns = df_mid.columns.to_list()[:8]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')
df_mid

,ms_drg,description,setting,rev_code,code,modifiers,ndc,drug_hcpcs_multiplier,payer_name,standard_charge
0,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,Inpatient,NaN,003,None,NaN,NaN,Gross Charge,1399304.2366
1,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,Inpatient,NaN,004,None,NaN,NaN,Gross Charge,908244.2900
2,020,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,Inpatient,NaN,020,None,NaN,NaN,Gross Charge,582781.5800
3,023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,Inpatient,NaN,023,None,NaN,NaN,Gross Charge,294478.6000
4,024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,Inpatient,NaN,024,None,NaN,NaN,Gross Charge,151325.3124
...,...,...,...,...,...,...,...,...,...,...
15003,981,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,981,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,55056.51
15004,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,982,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,58884.45
15005,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,983,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,4648.00
15006,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Inpatient,NaN,987,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,47451.02


In [20]:
set(df_mid['setting'].to_list())

{'Inpatient'}

In [21]:
df_mid['additional_generic_notes'] = df_mid['setting']
df_mid['billing_class'] = None
df_mid.loc[df_mid['setting'] == 'ProFee', 'billing_class'] = 'professional'
df_mid['setting'] = None
df_mid.loc[df_mid['additional_generic_notes'] == 'Inpatient', 'setting'] = 'inpatient'
df_mid.loc[df_mid['additional_generic_notes'] == 'Outpatient', 'setting'] = 'outpatient'
df_mid

,ms_drg,description,setting,rev_code,code,modifiers,ndc,drug_hcpcs_multiplier,payer_name,standard_charge,additional_generic_notes,billing_class
0,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,inpatient,NaN,003,None,NaN,NaN,Gross Charge,1399304.2366,Inpatient,None
1,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,inpatient,NaN,004,None,NaN,NaN,Gross Charge,908244.2900,Inpatient,None
2,020,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,inpatient,NaN,020,None,NaN,NaN,Gross Charge,582781.5800,Inpatient,None
3,023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,inpatient,NaN,023,None,NaN,NaN,Gross Charge,294478.6000,Inpatient,None
4,024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,inpatient,NaN,024,None,NaN,NaN,Gross Charge,151325.3124,Inpatient,None
...,...,...,...,...,...,...,...,...,...,...,...,...
15003,981,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,981,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,55056.51,Inpatient,None
15004,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,982,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,58884.45,Inpatient,None
15005,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,983,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,4648.00,Inpatient,None
15006,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,NaN,987,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,47451.02,Inpatient,None


In [22]:
def payer_name_to_payer_category(payer_name):
    if payer_name == 'Discounted Cash Price':
        return 'cash'
    elif payer_name == 'Gross Charge':
        return 'gross'
    elif payer_name == 'De-identified maximum negotiated charge':
        return 'max'
    elif payer_name == 'De-identified minimum negotiated charge':
        return 'min'

    return 'payer'

df_mid['payer_category'] = df_mid['payer_name'].apply(payer_name_to_payer_category)
df_mid

,ms_drg,description,setting,rev_code,code,modifiers,ndc,drug_hcpcs_multiplier,payer_name,standard_charge,additional_generic_notes,billing_class,payer_category
0,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,inpatient,NaN,003,None,NaN,NaN,Gross Charge,1399304.2366,Inpatient,None,gross
1,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,inpatient,NaN,004,None,NaN,NaN,Gross Charge,908244.2900,Inpatient,None,gross
2,020,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,inpatient,NaN,020,None,NaN,NaN,Gross Charge,582781.5800,Inpatient,None,gross
3,023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,inpatient,NaN,023,None,NaN,NaN,Gross Charge,294478.6000,Inpatient,None,gross
4,024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,inpatient,NaN,024,None,NaN,NaN,Gross Charge,151325.3124,Inpatient,None,gross
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15003,981,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,981,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,55056.51,Inpatient,None,payer
15004,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,982,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,58884.45,Inpatient,None,payer
15005,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,983,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,4648.00,Inpatient,None,payer
15006,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,NaN,987,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,47451.02,Inpatient,None,payer


In [23]:
df_mid = df_mid[df_mid['standard_charge'].notnull()]
df_mid.shape

(14995, 13)

In [24]:
df_mid = pd.DataFrame(df_mid) # XXX

if not 'hcpcs_cpt' in df_mid.columns:
    df_mid['hcpcs_cpt'] = None

df_mid.loc[df_mid['hcpcs_cpt'].isnull(), 'hcpcs_cpt'] = ''
df_mid.loc[df_mid['hcpcs_cpt'].str.isalpha(), 'code'] = df_mid[df_mid['hcpcs_cpt'].str.isalpha()]['hcpcs_cpt']
df_mid['hcpcs_cpt'] = df_mid['hcpcs_cpt'].apply(lambda cpt: '' if len(cpt) != 5 else cpt)
df_mid.loc[df_mid['hcpcs_cpt'].str.isalpha(), 'hcpcs_cpt'] = None
df_mid.loc[df_mid['hcpcs_cpt'] == '', 'hcpcs_cpt'] = None
set(df_mid['hcpcs_cpt'].to_list())

{None}

In [25]:
df_mid['hospital_id'] = ccn
df_mid['line_type'] = None
if not 'local_code' in df_mid.columns:
    df_mid['local_code'] = None
if not 'code' in df_mid.columns:
    df_mid['code'] = None
if not 'ms_drg' in df_mid.columns:
    df_mid['ms_drg'] = None
df_mid['apr_drg'] = None
df_mid['eapg'] = None
df_mid['alt_hcpcs_cpt'] = None
df_mid['thru'] = None
df_mid['apc'] = None
if not 'icd' in df_mid.columns:
    df_mid['icd'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['plan_name'] = None
df_mid['standard_charge_percent'] = None
df_mid['contracting_method'] = None
df_mid['additional_payer_specific_notes'] = None

df_mid

,ms_drg,description,setting,rev_code,code,modifiers,ndc,drug_hcpcs_multiplier,payer_name,standard_charge,...,thru,apc,icd,drug_quantity,drug_unit_of_measurement,drug_type_of_measurement,plan_name,standard_charge_percent,contracting_method,additional_payer_specific_notes
0,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,inpatient,NaN,003,None,NaN,NaN,Gross Charge,1399304.2366,...,None,None,None,None,None,None,None,None,None,None
1,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,inpatient,NaN,004,None,NaN,NaN,Gross Charge,908244.2900,...,None,None,None,None,None,None,None,None,None,None
2,020,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,inpatient,NaN,020,None,NaN,NaN,Gross Charge,582781.5800,...,None,None,None,None,None,None,None,None,None,None
3,023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,inpatient,NaN,023,None,NaN,NaN,Gross Charge,294478.6000,...,None,None,None,None,None,None,None,None,None,None
4,024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,inpatient,NaN,024,None,NaN,NaN,Gross Charge,151325.3124,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15003,981,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,981,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,55056.51,...,None,None,None,None,None,None,None,None,None,None
15004,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,982,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,58884.45,...,None,None,None,None,None,None,None,None,None,None
15005,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,inpatient,NaN,983,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,4648.00,...,None,None,None,None,None,None,None,None,None,None
15006,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,inpatient,NaN,987,None,NaN,NaN,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,47451.02,...,None,None,None,None,None,None,None,None,None,None


In [26]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,050455,None,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,NaN,None,003,003,None,None,None,...,None,inpatient,gross,Gross Charge,None,1399304.2366,None,None,Inpatient,None
1,050455,None,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,NaN,None,004,004,None,None,None,...,None,inpatient,gross,Gross Charge,None,908244.2900,None,None,Inpatient,None
2,050455,None,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,NaN,None,020,020,None,None,None,...,None,inpatient,gross,Gross Charge,None,582781.5800,None,None,Inpatient,None
3,050455,None,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,NaN,None,023,023,None,None,None,...,None,inpatient,gross,Gross Charge,None,294478.6000,None,None,Inpatient,None
4,050455,None,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,NaN,None,024,024,None,None,None,...,None,inpatient,gross,Gross Charge,None,151325.3124,None,None,Inpatient,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15003,050455,None,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,NaN,None,981,981,None,None,None,...,None,inpatient,payer,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,None,55056.51,None,None,Inpatient,None
15004,050455,None,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,NaN,None,982,982,None,None,None,...,None,inpatient,payer,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,None,58884.45,None,None,Inpatient,None
15005,050455,None,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,NaN,None,983,983,None,None,None,...,None,inpatient,payer,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,None,4648.00,None,None,Inpatient,None
15006,050455,None,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,NaN,None,987,987,None,None,None,...,None,inpatient,payer,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS,None,47451.02,None,None,Inpatient,None


In [27]:
df_out.to_csv("tmp.csv", index=False)

In [28]:
def convert_chunk(chunk, ccn):
    csv_buf = StringIO(chunk)

    df_in = pd.read_csv(csv_buf, dtype=str)

    df_mid = pd.DataFrame(df_in)
    df_mid = df_mid.rename(columns={
        'Procedure Code': 'local_code',
        'Procedure Description': 'description',
        'Price Tier': 'setting',
        'Revenue Code': 'rev_code',
        'CPT HCPCS Code': 'hcpcs_cpt',
        'NDC Code': 'ndc',
        'Rx Unit Multiplier': 'drug_hcpcs_multiplier',
        'Modifier1': 'modifiers',
        'Diagnosis Related Group Code': 'ms_drg',
        'Diagnosis Related Group Description': 'description',
        'CPT HCPCS DRG Code': 'code',
        'Shoppable Services Code': 'local_code',
        'Shoppable Services Description': 'description'
    })

    df_mid.loc[df_mid['modifiers'].isnull(), 'modifiers'] = None
    df_mid.loc[df_mid['Modifier2'].isnull(), 'Modifier2'] = None

    df_mid['modifiers'] = df_mid[['modifiers', 'Modifier2']].apply(lambda row: unify_modifiers(row['modifiers'], row['Modifier2']), axis=1)
    del df_mid['Modifier2']

    remaining_columns = df_mid.columns.to_list()[:8]
    df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')

    df_mid['additional_generic_notes'] = df_mid['setting']
    df_mid['billing_class'] = None
    df_mid.loc[df_mid['setting'] == 'ProFee', 'billing_class'] = 'professional'
    df_mid['setting'] = None
    df_mid.loc[df_mid['additional_generic_notes'] == 'Inpatient', 'setting'] = 'inpatient'
    df_mid.loc[df_mid['additional_generic_notes'] == 'Outpatient', 'setting'] = 'outpatient'

    df_mid['payer_category'] = df_mid['payer_name'].apply(payer_name_to_payer_category)

    df_mid = df_mid[df_mid['standard_charge'].notnull()]

    df_mid = pd.DataFrame(df_mid) # XXX

    if not 'hcpcs_cpt' in df_mid.columns:
        df_mid['hcpcs_cpt'] = None

    df_mid.loc[df_mid['hcpcs_cpt'].isnull(), 'hcpcs_cpt'] = ''
    df_mid.loc[df_mid['hcpcs_cpt'].str.isalpha(), 'code'] = df_mid[df_mid['hcpcs_cpt'].str.isalpha()]['hcpcs_cpt']
    df_mid['hcpcs_cpt'] = df_mid['hcpcs_cpt'].apply(lambda cpt: '' if len(cpt) != 5 else cpt)
    df_mid.loc[df_mid['hcpcs_cpt'].str.isalpha(), 'hcpcs_cpt'] = None
    df_mid.loc[df_mid['hcpcs_cpt'] == '', 'hcpcs_cpt'] = None

    df_mid['hospital_id'] = ccn
    df_mid['line_type'] = None
    if not 'local_code' in df_mid.columns:
        df_mid['local_code'] = None
    if not 'code' in df_mid.columns:
        df_mid['code'] = None
    if not 'ms_drg' in df_mid.columns:
        df_mid['ms_drg'] = None
    df_mid['apr_drg'] = None
    df_mid['eapg'] = None
    df_mid['alt_hcpcs_cpt'] = None
    df_mid['thru'] = None
    df_mid['apc'] = None
    if not 'icd' in df_mid.columns:
        df_mid['icd'] = None
    df_mid['drug_quantity'] = None
    df_mid['drug_unit_of_measurement'] = None
    df_mid['drug_type_of_measurement'] = None
    df_mid['plan_name'] = None
    df_mid['standard_charge_percent'] = None
    df_mid['contracting_method'] = None
    df_mid['additional_payer_specific_notes'] = None

    df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
    return df_out

In [29]:
def perform_task(h_f, ccn, app_url, transparency_page):
    resp = requests.get(app_url)
    print(resp.url)

    tree = html.fromstring(resp.text)
    js_link = tree.xpath('//script[contains(@src, "/PTT/extnet/extnet-init-js")]/@src')[0]
    js_url = urljoin(resp.url, js_link)
    resp1 = requests.get(js_url)
    parsed = js2xml.parse(resp1.text)
    db_name = parsed.xpath('//object[./property/string[text()="App.hdnDB_Container"]]/property[@name="value"]/string/text()')[0]

    params = { 'dbName': db_name, 'type': 'CDMWithoutLabel' }

    data = '------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__EVENTTARGET"\r\n\r\nResourceManager\r\n------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__EVENTARGUMENT"\r\n\r\n-|public|DownloadReport\r\n------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__ExtNetDirectEventMarker"\r\n\r\ndelta=true\r\n------WebKitFormBoundarysVqstz3xq11k5yBT--\r\n'

    resp2 = requests.post('https://apps.para-hcfs.com/PTT/FinalLinks/Reports.aspx', params=params, data=data)
    print(resp2.url)

    dfs = []
    
    chunks = resp2.text.split("\r\n\r\n")

    for chunk in chunks[1:]:
        df_tmp = convert_chunk(chunk, ccn)
        dfs.append(df_tmp)

    df_out = pd.concat(dfs)

    df_out.to_csv('rate_' + ccn + '.csv', index=False)
    
    filename = resp2.headers['content-disposition'].split('"')[1]
    ein = derive_ein_from_filename(filename)
    
    date_str = chunks[0].split(" ")[-1]

    last_updated_at = parse_datetime(date_str).isoformat().split("T")[0]
    query = 'UPDATE hospital SET ein = "{}", last_updated = "{}", file_name = "{}", mrf_url = "{}", transparency_page = "{}" WHERE id = "{}";'.format(
        ein, last_updated_at, filename, app_url, transparency_page, ccn)

    h_f.write(query)
    h_f.write("\n")

In [30]:
h_f = open("hospitals.sql", "w")
perform_task(h_f, ccn, app_url, transparency_page)
h_f.close()

https://apps.para-hcfs.com/PTT/FinalLinks/Adventist_Bakersfield_V3.aspx
https://apps.para-hcfs.com/PTT/FinalLinks/Reports.aspx?dbName=dbAHBBAKERSFIELDCA&type=CDMWithoutLabel
